In [0]:
# training code for SQuAD 2.0 dataset

In [2]:
# TODO(lanzhzh): Add support for 2.x.
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf

assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = "2x2"
print("TPU address is", TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.

TensorFlow 1.x selected.
TPU address is grpc://10.52.123.130:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1604170493049168519),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2977581456294318553),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4416960975892318300),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 185515700976686323),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14150346335667952570),
 _DeviceAttributes(/job:tpu_

In [3]:
import sys

!test -d albert || git clone https://github.com/google-research/albert albert
if not 'albert' in sys.path:
  sys.path += ['albert']
  
!pip install sentencepiece

Cloning into 'albert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 226.59 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (225/225), done.
     |████████████████████████████████| 1.0MB 3.3MB/s 


In [4]:
BUCKET = "albert-train-squad" 
TASK = "SQuAD_2.0"
# Available pretrained model checkpoints:
#   base, large, xlarge, xxlarge
ALBERT_MODEL = 'base' 

TASK_DATA_DIR = 'squad_2.0_data'

print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://albert-train-squad/albert-tfhub/squad_models/SQuAD_2.0 *****


In [5]:
# Download squad data.
 TASK_DATA_DIR = 'squad_2.0_data'
 !mkdir $TASK_DATA_DIR \
 && cd $TASK_DATA_DIR \
 && wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
 && wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

# ! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
# !python download_glue_repo/download_glue_data.py --data_dir=$TASK_DATA_DIR --tasks=$TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))

# ALBERT_MODEL_HUB = 'https://tfhub.dev/google/albert_' + ALBERT_MODEL + '/3'

--2020-04-12 15:18:25--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  55.2MB/s    in 0.7s    

2020-04-12 15:18:26 (55.2 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-04-12 15:18:26--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [6]:
!python albert/modeling.py



!pip install -r albert/requirements.txt

     |████████████████████████████████| 92kB 2.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=aa16dd7194171bbc4789a9ef8d50f59e0055767ce5499cec0f23f04579ddad01
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [0]:
# os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs"\
  --train_file="squad_2.0_data/train-v2.0.json"\
  --predict_file="squad_2.0_data/dev-v2.0.json"\
  --vocab_file="albert_base/30k-clean.vocab" \
  --init_checkpoint="albert_base/model.ckpt-best" \
  --spm_model_file="albert_base/30k-clean.model"\
  --do_lower_case \
  --max_seq_length=384\
  --doc_stride=128\
  --max_query_length=64 \
  --do_predict \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30\
  --train_feature_file="SQuAD_train_feature_file.tf" \
  --predict_feature_file="SQuAD_predict_feature_file.tf" \
  --predict_feature_left_file="SQuAD_predict_left_feature_file.tf" \
  --do_train \
  

INFO:tensorflow:loading sentence piece model
I0411 19:16:17.513501 139922025551744 tokenization.py:188] loading sentence piece model
W0411 19:16:19.740746 139922025551744 estimator.py:1994] Estimator's model_fn (<function v2_model_fn_builder.<locals>.model_fn at 0x7f41e256ff28>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'albert/outputs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <te

In [10]:
!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs" \
  --train_file="squad_2.0_data/train-v2.0.json" \
  --predict_file="squad_2.0_data/dev-v2.0.json" \
  --vocab_file="albert_base/30k-clean.vocab" \
  --train_feature_file="SQuAD_train_feature_file.tf" \
  --predict_feature_file="SQuAD_predict_feature_file.tf" \
  --predict_feature_left_file="SQuAD_predict_left_feature_file.tf" \
  --spm_model_file="albert_base/30k-clean.model" \
  --albert_hub_module_handle=https://tfhub.dev/google/albert_base/1\
  --do_train\
  --do_lower_case \
  --max_seq_length=384 \
  --doc_stride=128 \
  --max_query_length=64 \
  --do_predict \
  --use_gpu \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30 
    # https://tfhub.dev/google/albert_base/1
    #  --init_checkpoint="albert_base/model.ckpt-best.index" \
    #   --do_train \

I0412 15:39:09.856758 140223124940672 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I0412 15:39:11.522926 140223124940672 saver.py:1503] Saver not created because there are no variables in the graph to restore
2020-04-12 15:39:11.597960: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-12 15:39:11.599830: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-12 15:39:11.599871: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b33c378d5483): /proc/driver/nvidia/version does not exist
2020-04-12 15:39:11.605607: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-12 15:39:11.605807: I tensorflow/compiler/xla/service/servi

In [0]:
!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs"\
  --train_file="squad_2.0_data/train-v2.0.json"\
  --predict_file="squad_2.0_data/dev-v2.0.json"\
  --vocab_file="albert_base/30k-clean.vocab" \
  --albert_hub_module_handle=https://tfhub.dev/google/albert_base/1\
  --spm_model_file="albert_base/30k-clean.model"\
  --do_lower_case \
  --max_seq_length=384\
  --doc_stride=128\
  --max_query_length=64 \
  --do_predict \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30\
  --train_feature_file="SQuAD_train_feature_file.tf" \
  --predict_feature_file="SQuAD_predict_feature_file.tf" \
  --predict_feature_left_file="SQuAD_predict_left_feature_file.tf" \

I0412 15:42:58.671257 139834999228288 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I0412 15:43:00.400471 139834999228288 saver.py:1503] Saver not created because there are no variables in the graph to restore
2020-04-12 15:43:00.504537: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-12 15:43:00.561699: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-12 15:43:00.561773: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b33c378d5483): /proc/driver/nvidia/version does not exist
2020-04-12 15:43:00.585319: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-12 15:43:00.585753: I tensorflow/compiler/xla/service/servi

In [0]:
from google.colab import drive
drive.mount('/content/drive')